In [7]:
import pandas as pd
import numpy as np
from collections import Counter

In [8]:
df_game = pd.read_csv('csvs/game.csv')[['match_id','game_id','game_nbr','score_w',
                                        'score_l','w_team_id','l_team_id','skill_lvl']]
df_game['game_id'] = [int(x[1:]) for x in df_game.game_id]
df_game['match_id'] = [int(x[1:]) for x in df_game.match_id]
df_game = df_game.sort_values(['match_id','game_id']).reset_index(drop=True)
print('GAME:\n ')
df_game.head(2)

GAME:
 


,match_id,game_id,game_nbr,score_w,score_l,w_team_id,l_team_id,skill_lvl
0,1,1,1,12,10,T2,T1,Pro
1,1,2,2,11,7,T1,T2,Pro


In [9]:
df_rally = pd.read_csv('csvs/rally.csv')
df_rally = df_rally[['match_id', 'game_id', 'rally_id', 'point_nbr', 'srv_team_id',
                     'rtrn_team_id', 'w_team_id', 'to_ind', 'to_team_id', 'ending_type']]
df_rally['match_id'] = [int(x[1:]) for x in df_rally.match_id]
df_rally['game_id'] = [int(x[1:]) for x in df_rally.game_id]
df_rally['rally_id'] = [int(x[1:]) for x in df_rally.rally_id]
df_rally = df_rally.sort_values(
    ['match_id', 'game_id', 'rally_id', 'point_nbr']).reset_index(drop=True)
print('RALLY:\n')
df_rally.head(2)

RALLY:



,match_id,game_id,rally_id,point_nbr,srv_team_id,rtrn_team_id,w_team_id,to_ind,to_team_id,ending_type
0,1,1,1,1,T1,T2,T2,N,NaN,Other
1,1,1,2,2,T2,T1,T2,N,NaN,Winner


In [10]:
def generate_streak_info(df,strkcol):
    """
    Parameters
    ----------
    
    df:
        A homogenous dataframe, 
        i.e. contains only the data germaine to the desired streak  
        (e.g. all rallys for a single game) 
        and be pre-sorted chronologically
    strkcol:
        Column within df with 2 unique values 
        
    Returns
    -------

    streaks:
        df with a new column
        strk_{strkcol} containing tallies for each streak.
    """
    data = df[strkcol].to_frame()
    data['strk_init'] = data[strkcol].ne(data[strkcol].shift())
    data['strk_id'] = data.strk_init.cumsum()
    strk_field = f'strk_{strkcol}'
    data[strk_field] = data.groupby('strk_id').cumcount() + 1
    streaks = pd.concat([df, data[strk_field]], axis=1)
    return streaks

In [11]:
all_to_strks=[]
for i in df_game.game_id:
    dfi = df_rally[df_rally.game_id==i]
    dfi = generate_streak_info(dfi,'to_ind')
    all_to_strks.append(list(dfi.strk_to_ind))
all_to_strks = list(np.concatenate(all_to_strks).flat)
df_rally['to_streaks'] = all_to_strks

In [12]:
#df_rally[df_rally.game_id==1].to_csv('TO_streaks_game1.csv')


In [14]:
len(df_rally)

2043

In [15]:
len(df_game)

43